<a href="https://colab.research.google.com/github/aimanh22/706_Assignments/blob/main/GPU_Example_Movie_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup

In [115]:
#@title Install
!pip install numba
!find / -iname 'libdevice'
!find / -iname 'libnvvm.so'
!nvidia-smi

/usr/local/cuda-10.1/nvvm/libdevice
/usr/local/cuda-10.0/nvvm/libdevice
/usr/local/cuda-10.1/nvvm/lib64/libnvvm.so
/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so
Sat Oct 24 17:37:34 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    29W /  70W |    701MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+---------

Headers

In [132]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import math
from numba import jit, njit, vectorize, cuda, uint32, f8, uint8, float32,float64
from pylab import imshow, show
from timeit import default_timer as timer
import nltk
import pandas as pd
from nltk.corpus import stopwords

In [117]:
print(cuda.gpus)

<Managed Device 0>


In [118]:
nltk.download('movie_reviews')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [119]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

porter    = nltk.PorterStemmer()

Files

In [120]:
import glob

# read in reviews from the NLTK corpora directory
datadir = '/root/nltk_data/corpora/movie_reviews'
neg     = datadir + '/neg/'
pos     = datadir + '/pos/'

posarr  = []
negarr  = []

pfiles = [f for f in glob.glob(pos + "*.txt", recursive=False)]
nfiles = [f for f in glob.glob(neg + "*.txt", recursive=False)]
for f in pfiles:
    with open(f) as my_file:
        posarr.append(my_file.read().replace('\n',''))
for f in nfiles:
    with open(f) as my_file:
        negarr.append(my_file.read().replace('\n',''))

In [122]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
document_list=posarr+negarr
document_list[:2]
document_list=document_list[:10]

In [123]:
document_list= [nltk.tokenize.word_tokenize(sent) for sent in document_list]
print(document_list[0])
for i in range(len(document_list)):
    #Converting the words into lower case
    document_list[i]=[w.lower() for w in document_list[i]]
    #Removing the stopwords
    document_list[i]=[w for w in document_list[i] if not str(w) in stop_words]
    #Retain only alphabet and remove everything else- punctuation and numbers
    document_list[i]=[w for w in document_list[i] if w.isalpha()]
document_list[0]

['when', 'jim', 'henson', 'passed', 'away', ',', 'he', 'left', 'behind', 'diverse', 'legions', 'of', 'fans', 'and', 'a', 'company', 'whose', 'ultimate', 'success', ',', 'it', 'now', 'seems', ',', 'hinged', 'on', 'his', 'input', '.', 'jim', 'henson', 'productions', 'and', 'the', 'creature', 'shop', 'are', 'still', 'thriving', 'financially', ',', 'but', 'as', 'the', 'last', 'two', 'muppet', 'films', '(', 'or', 'that', 'silly', 'computer-generated', 'monkey', 'from', 'lost', 'in', 'space', ')', 'demonstrate', ',', 'the', 'thrill', 'and', 'genius', 'are', 'gone', '.', 'i', "'m", 'not', 'sure', 'the', 'dark', 'crystal', ',', 'made', 'today', ',', 'would', 'generate', 'from', 'an', 'audience', 'of', 'kids', 'five', 'to', 'fifty', 'the', 'same', 'awe-filled', 'response', '.', 'an', 'all-powerful', 'crystal', 'has', 'cracked', ',', 'causing', 'the', 'leaders', 'of', 'the', 'green', 'world', 'to', 'split', 'apart', 'into', 'two', 'beings', ':', 'the', 'big', ',', 'gentle', 'mystics', ',', 'and'

['jim',
 'henson',
 'passed',
 'away',
 'left',
 'behind',
 'diverse',
 'legions',
 'fans',
 'company',
 'whose',
 'ultimate',
 'success',
 'seems',
 'hinged',
 'input',
 'jim',
 'henson',
 'productions',
 'creature',
 'shop',
 'still',
 'thriving',
 'financially',
 'last',
 'two',
 'muppet',
 'films',
 'silly',
 'monkey',
 'lost',
 'space',
 'demonstrate',
 'thrill',
 'genius',
 'gone',
 'sure',
 'dark',
 'crystal',
 'made',
 'today',
 'would',
 'generate',
 'audience',
 'kids',
 'five',
 'fifty',
 'response',
 'crystal',
 'cracked',
 'causing',
 'leaders',
 'green',
 'world',
 'split',
 'apart',
 'two',
 'beings',
 'big',
 'gentle',
 'mystics',
 'skeksis',
 'mystics',
 'send',
 'jen',
 'naive',
 'gelfling',
 'boy',
 'mission',
 'find',
 'shard',
 'cracked',
 'away',
 'must',
 'reinserted',
 'crystal',
 'skeksis',
 'become',
 'eternal',
 'rulers',
 'great',
 'conjunction',
 'three',
 'suns',
 'along',
 'way',
 'jen',
 'encounters',
 'teams',
 'gelfling',
 'alive',
 'rebecca',
 'kira',

In [124]:
vocab = set()
document_list=[[porter.stem(w) for w in word_list] for word_list in document_list]
#List conatining the number of words in each document 
sizearr={ind:len(list(set(document_list[ind]))) for ind in range(len(document_list))}
#Setting Vocabulary as all unique words in the list
vocab=set(sum(document_list,[]))
# your code for creating the dataframe goes here.
# the resulting dataframe should be called "df"
#
# Each row of the dataframe should represent a document (inaugural address)
# Each column of the dataframe should be a term from the vocab
df = pd.DataFrame()
for doc in document_list: #For each document
        dict_word_count_document=dict() #create an empty dict
        for word in vocab:
            dict_word_count_document[word]=doc.count(word) #update dict with word: count(word)
            pass
        df=df.append(dict_word_count_document,ignore_index=True) #update dataframe
        pass

In [125]:
df

,abbott,abil,abl,aboard,absolut,absorb,abus,accept,accord,achiev,act,action,activ,actor,actress,actual,ad,adam,adapt,address,admit,ador,adrian,advantag,advisor,affect,afraid,afterglow,afterlif,agenda,agent,agre,akira,alessandro,alex,alexand,alien,aliv,alleg,allen,...,willi,william,win,wind,wing,wipe,wish,wit,witherspoon,without,witti,wittliff,wolf,wolfgang,woman,wonder,woodi,word,work,worker,world,worst,worth,would,wrap,wreak,write,writer,written,wrong,wwii,ye,year,yearbook,yellow,yet,york,young,younger,zorg
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,3.0,1.0,0.0
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,1.0


In [133]:
def computetf(wordvec,doclen):
    tf=dict() #create an empty dict
    for word in wordvec.index:
        tf[word]=wordvec[word]/doclen #append termfreq of word (in the document) in the dict
        pass
    return tf
def computeidf(df):
    idf=dict() #create empty dict
    for word in vocab:
        number_of_docs_with_word=len(df[df[word]>0])
        total_number_of_docs=len(df)
        idf[word]=math.log(total_number_of_docs/number_of_docs_with_word)#append inversetermfreq of the docs
        pass
    return idf        

In [134]:
newdf = pd.DataFrame()

idfdict = computeidf(df)

cols = df.columns
for index, row in df.iterrows():
    newrow = computetf(row,sizearr[index])
    for c in cols:
        newrow[c] = newrow[c]*idfdict[c]
    newdf = newdf.append(newrow,ignore_index=True)

In [147]:
from numba import (cuda, vectorize,guvectorize)
import pandas as pd
import numpy as np

from functools import wraps
from time import time

def doc_array():
    rea = newdf.to_numpy()
    return np.float64(rea)


@vectorize(['float64(float64, float64)'], target='cuda')
def mul_ufunc(x, y):
    return x*y

@guvectorize(['(float64[:],float64)'],'(n)->()', target='cuda')
def add_ufunc(x,res):
    i=0
    for a in x:
      i=i+a
    res=i
    return res

@guvectorize(['(float64[:],float64)'],'(n)->()', target='cuda')
def cal_ufunc(x,res):
    i=0
    for a in x:
      i=i+a*a
    res=i

@vectorize(['float64(float64, float64,float64)'], target='cuda')
def dist_ufunc(d, x, y):
    if x!=0 and y!=0:
      return d/x*y
    else:
      return 0

@guvectorize(['(float64[:],float64[:],float64)'],'(n),(n)->()', target='cuda')
def distance(x,y,res):
    res=10
   
def cuda_operation():
    """Performs Vectorized Operations on GPU"""
    x=doc_array()
    y=doc_array()
    
    print("Moving calculations to GPU memory")
    x_device = cuda.to_device(x)
    y_device = cuda.to_device(y)
    out_device = cuda.device_array(
        shape=(x_device.shape[0],x_device.shape[1]), dtype=np.float64)
    print(x_device)
    print(x_device.shape)
    print(x_device.dtype)

    print("Calculating on GPU: The Multiplication")
    mul_ufunc(x_device,y_device, out=out_device)
    out_host = out_device.copy_to_host()
    mul_val=out_host
    print(mul_val)
    
    mul_device=cuda.to_device(mul_val)
    #print(mul_device)
    out_device2 = cuda.device_array(
        shape=(mul_device.shape[0],), dtype=np.float64)
    #print(mul_device)
    #print(mul_device.shape)
    #print(mul_device.dtype)
    #print(out_device2.shape)
    #print(sum(out_device2))
    add_ufunc(mul_val,out=out_device2)
    print(out_device2)
    a = out_device2.copy_to_host()
    print(f"Calculations from GPU {a}")
    print(sum(a))
    cal_ufunc(x_device,out=out_device2)
    x = out_device2.copy_to_host()
     
    cal_ufunc(y_device,out=out_device2)
    y = out_device2.copy_to_host()

    print(x) 
    print(y)
    
    d=dist_ufunc(x,y,a)
    print(a)
cuda_operation()

Moving calculations to GPU memory
(10, 1478)
float64
Calculating on GPU: The Multiplication
[[0.0000000000e+00 0.0000000000e+00 0.0000000000e+00 ... 0.0000000000e+00
  0.0000000000e+00 0.0000000000e+00]
 [0.0000000000e+00 0.0000000000e+00 0.0000000000e+00 ... 0.0000000000e+00
  0.0000000000e+00 0.0000000000e+00]
 [0.0000000000e+00 0.0000000000e+00 0.0000000000e+00 ... 0.0000000000e+00
  0.0000000000e+00 0.0000000000e+00]
 ...
 [0.0000000000e+00 0.0000000000e+00 0.0000000000e+00 ... 6.7427384266e-05
  0.0000000000e+00 0.0000000000e+00]
 [0.0000000000e+00 7.6651362756e-05 0.0000000000e+00 ... 0.0000000000e+00
  0.0000000000e+00 0.0000000000e+00]
 [0.0000000000e+00 0.0000000000e+00 0.0000000000e+00 ... 0.0000000000e+00
  0.0000000000e+00 8.4830369768e-05]]
Calculations from GPU [0.0000000000e+00 8.4830369768e-05 0.0000000000e+00 0.0000000000e+00
 0.0000000000e+00 0.0000000000e+00 4.1750850863e-06 0.0000000000e+00
 0.0000000000e+00 0.0000000000e+00]
8.900545485398899e-05
[0.0000000000e+00 

In [148]:
sum(doc_array()[0])

2.2023114486150277

In [150]:
def cuda_operation_dist():
    x=doc_array()
    #y=doc_array()
    y=np.repeat(doc_array(),x.shape[0])
    y=y.reshape(x.shape[0],x.shape[0],x.shape[1])
    #y=y.T
    print("Moving calculations to GPU memory")
    x_device = cuda.to_device(x)
    y_device = cuda.to_device(y)
    #print(x)
    #print(y)
    print(x_device.shape)
    print(y_device.shape)
    print(sum(x))
    print(sum(y))
    out_device = cuda.device_array(
        shape=(x_device.shape[0],), dtype=np.float64)
    print(out_device.shape)
    for ele in y:
      ele_device=cuda.to_device(ele)
      d=distance(x_device,ele_device)
      print(d)
      out_host = out_device.copy_to_host()
      print("The distances matrix")
      print(out_host)
cuda_operation_dist()

Moving calculations to GPU memory
(10, 1478)
(10, 10, 1478)
[0.0082235182 0.0087550764 0.0082235182 ... 0.0255794529 0.0082826802
 0.0092103404]
[[0.0082235182 0.0082235182 0.0082235182 ... 0.0114556472 0.0114556472
  0.0114556472]
 [0.0114556472 0.0114556472 0.0143911568 ... 0.0143911568 0.0143911568
  0.0143911568]
 [0.0143911568 0.0143911568 0.0143911568 ... 0.0082826802 0.0082826802
  0.0082826802]
 ...
 [0.0204923664 0.0204923664 0.0204923664 ... 0.0228209141 0.0228209141
  0.0228209141]
 [0.0228209141 0.0228209141 0.0228209141 ... 0.0180661407 0.0114556472
  0.0114556472]
 [0.0114556472 0.0114556472 0.0114556472 ... 0.0092103404 0.0092103404
  0.0092103404]]
(10,)
The distances matrix
[0.0000000000e+00 8.4830369768e-05 0.0000000000e+00 0.0000000000e+00
 0.0000000000e+00 0.0000000000e+00 4.1750850863e-06 0.0000000000e+00
 0.0000000000e+00 0.0000000000e+00]
The distances matrix
[0.0000000000e+00 8.4830369768e-05 0.0000000000e+00 0.0000000000e+00
 0.0000000000e+00 0.0000000000e+00 4